In [73]:
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [74]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [75]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [76]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [77]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [101.,  44.,  37.],
        [ 68.,  97.,  70.],
        [ 91.,  88.,  64.],
        [ 88., 134.,  59.]])
tensor([[ 80., 102.],
        [ 21.,  38.],
        [102., 120.],
        [ 81., 101.],
        [118., 132.]])


In [78]:
# Defining a Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3710,  0.3287, -0.4189],
        [ 0.3472,  0.3483,  0.4933]], requires_grad=True)
Parameter containing:
tensor([ 0.1720, -0.1227], requires_grad=True)


In [79]:
predict = model(inputs)
print(predict)

tensor([[ 31.2649,  69.7740],
        [ 36.0487,  93.6984],
        [ 52.1989, 105.3706],
        [ 36.6476,  68.5248],
        [ 28.0035,  91.8057],
        [ 31.3071,  69.7730],
        [ 35.3011,  93.8435],
        [ 52.1510, 106.2111],
        [ 36.6053,  68.5258],
        [ 27.2136,  91.9518],
        [ 30.5173,  69.9191],
        [ 36.0910,  93.6974],
        [ 52.9465, 105.2255],
        [ 37.4374,  68.3787],
        [ 27.9612,  91.8068]], grad_fn=<AddmmBackward>)


In [80]:
?nn.Linear

Init signature: nn.Linear(in_features: int, out_features: int, bias: bool = True) -> None
Docstring:     
Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

Args:
    in_features: size of each input sample
    out_features: size of each output sample
    bias: If set to ``False``, the layer will not learn an additive bias.
        Default: ``True``

Shape:
    - Input: :math:`(N, *, H_{in})` where :math:`*` means any number of
      additional dimensions and :math:`H_{in} = \text{in\_features}`
    - Output: :math:`(N, *, H_{out})` where all but the last dimension
      are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

Attributes:
    weight: the learnable weights of the module of shape
        :math:`(\text{out\_features}, \text{in\_features})`. The values are
        initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`, where
        :math:`k = \frac{1}{\text{in\_featu

In [81]:
# Loss Function
loss = F.mse_loss(predict, targets)
print(loss)

tensor(1545.7784, grad_fn=<MseLossBackward>)


Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.


In [82]:
opt = torch.optim.SGD(model.parameters(),lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

In [83]:
def fit(no_epoch, model, loss_fn, opt):
    for epoch in range(no_epoch):
        for xb, yb in train_dl:
            predict = model(xb)
            loss = loss_fn(predict, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            # Print the progress
            if (epoch+1) % 10 == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, no_epoch, loss.item()))


In [84]:
fit(100,model,F.mse_loss, opt)

Epoch [10/100], Loss: 513.6397
Epoch [10/100], Loss: 605.7823
Epoch [10/100], Loss: 165.7368
Epoch [20/100], Loss: 387.4932
Epoch [20/100], Loss: 287.4246
Epoch [20/100], Loss: 232.1525
Epoch [30/100], Loss: 160.4231
Epoch [30/100], Loss: 146.5717
Epoch [30/100], Loss: 356.8982
Epoch [40/100], Loss: 207.7266
Epoch [40/100], Loss: 168.4710
Epoch [40/100], Loss: 106.9063
Epoch [50/100], Loss: 97.5026
Epoch [50/100], Loss: 185.7493
Epoch [50/100], Loss: 85.8614
Epoch [60/100], Loss: 47.8863
Epoch [60/100], Loss: 136.1805
Epoch [60/100], Loss: 95.5895
Epoch [70/100], Loss: 68.5815
Epoch [70/100], Loss: 74.1518
Epoch [70/100], Loss: 76.1277
Epoch [80/100], Loss: 57.8781
Epoch [80/100], Loss: 57.2032
Epoch [80/100], Loss: 70.1635
Epoch [90/100], Loss: 36.0238
Epoch [90/100], Loss: 45.3950
Epoch [90/100], Loss: 72.8807
Epoch [100/100], Loss: 37.5444
Epoch [100/100], Loss: 82.1230
Epoch [100/100], Loss: 14.4629
